# 该文件只是用来测试class之后会删

In [12]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [13]:
df_index = pd.read_excel("../../Data/Index.xls",index_col=0)
df_index = df_index.dropna()
df_corr_index = df_index.corr()

In [43]:
np.array(r_index.iloc[:,0])

r=r_index
r["p"] = r_p

cov_index_p=np.array([],dtype="float64")
for i in range(20):
    cov_index_p=np.append(cov_index_p, np.cov(r_index.iloc[1:,i],r_p[1:])[0, 1])
mrc = cov_index_p/r_p.std()

trc = w*mrc
trc_m = np.array([trc]*20)

x = [1,2,3,4]
x_m = np.array([x]*4)
x_m_t = x_m.T

x_m

((x_m-x_m_t)**2)

x_m_t

In [180]:
class RPModel:
    """
    Des: Risk-Parity Model
    
    Input:
        NA is not allowed
        Monthly frequence
        date(increasing): %Y/%m/%d
        history data: time series of history price of the assets
        transcation view: view of long, short or close a position
        risk-free rate: time series of risk-free rate
        initial_asset(=100): the value of initital asset
    """
    

    def __init__(self, df_date, df_history):
        self.df_date = df_date
        self.df_history = df_history
        self.num = df_history.shape[1]
        self.r_index = np.log(df_index/df_index.shift(1))
        
    
    def optimize(self, output = True):
        sigma_p = self.r_p.std()
        
        def Error_Min(w):
            # calculate Total Risk Contribution
            cov_index_p=np.array([],dtype="float64")
            r_p = (self.r_index*w).sum(axis=1)
            for i in range(self.num):
                cov_index_p = np.append(cov_index_p, np.cov(self.r_index.iloc[1:,i], r_p[1:])[0, 1])
            mrc = cov_index_p/sigma_p
            trc = w*mrc

            # calculate the deviation
            trc_m = np.array([trc]*self.num)
            trc_m_t = trc_m.T
            e = (trc_m-trc_m_t)**2
            return e.sum()
        
        cons = ({'type': 'eq', 'fun': lambda w: w.sum() - 1 })
        w = [1/self.num,]*self.num
        
        # optimization
        res = minimize(Error_Min,w,constraints=cons)
        
        # output
        if output:
            print("Status: "+str(res.message))
        
        return res.x

In [181]:
RPModel()

TypeError: __init__() missing 2 required positional arguments: 'df_date' and 'df_history'

In [192]:
num = 20
def Error_Min(w):
    # calculate Total Risk Contribution
    cov_index_p=np.array([],dtype="float64")
    r_p = (r_index*w).sum(axis=1)
    sigma_p=r_p.std()
    for i in range(num):
        cov_index_p = np.append(cov_index_p, np.cov(r_index.iloc[1:,i],r_p[1:])[0, 1])
    mrc = cov_index_p/sigma_p
    trc = w*mrc

    # calculate the deviation
    trc_m = np.array([trc]*num)
    trc_m_t = trc_m.T
    e = (trc_m-trc_m_t)**2
    return e.sum()

cons = ({'type': 'eq', 'fun': lambda w: w.sum() - 1 })
w = np.array([1/num,]*(num))

# optimization
res = minimize(Error_Min,w,constraints=cons, tol=1e-8)

In [194]:
res.x

array([0.00902367, 0.00511434, 0.00733612, 0.00518473, 0.00650393,
       0.00818454, 0.00960919, 0.00603608, 0.00679933, 0.20921808,
       0.21037603, 0.20926261, 0.2134494 , 0.04362414, 0.0060843 ,
       0.01002971, 0.00692089, 0.00749122, 0.00599547, 0.01375621])